# Введение в машинное обучение

## Семинар #7

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

code credit @artonson

## Обучение без учителя. Поиск аномалий (Anomaly Detection)

In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

## Кто такие аутлайеры (аномалии)? 

Это наблюдение, которое "**отличается**" "**настолько сильно**" от "**прочих наблюдений**"

##### Назовите самый простой способ детектирования аномалий?

*Спойлер: СТД? Квантили?*

## 1.  Ищем фрод

Источник: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
data = pd.read_csv("./data/creditcard.csv")

Давайте проверим, сколько "фрода" в этой выборке?

In [ ]:
data.head()

In [ ]:
data['Class'].value_counts()

In [ ]:
492/28432

Должно быть  1.7% , что можно сказать про баланс классов?

In [ ]:
data.describe()

In [ ]:
full_X = data.drop(columns="Class")
full_y = data["Class"]

In [ ]:
import seaborn as sns
sns.distplot(data["Amount"], bins = 100)

### Как распределены характеристики?

Будем делать **красиво**:
    
    - отложим заранее из оригинальной выборки тестовую, размером 0.4
    - будем выбирать модель кросс валидацией на выборке `train`
    - не забудем про стратификацию

In [ ]:
from sklearn.model_selection import train_test_split

full_X.reset_index(drop=True)

tt_split = train_test_split(full_X, full_y, test_size=0.4, stratify=full_y)

train_X, test_X, train_y, test_y = tt_split 

Будем делать **красиво**, выберем несколько метрик:
    
    1. стандартно `accuracy`, почему это плохо в нашем случае?
    2. `precision_score` или доля истинных 1 среди всех помеченных как 1
    3. `recall_score` или полнота "вероятность", с которой все истинные 1 предсказаны как 1
    4. `fbeta_score` - частный случай которого, наш знакомый `f1_score` 

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, fbeta_score, f1_score

Зачем нам $\beta$ ? : https://en.wikipedia.org/wiki/F1_score

Возьмем $\beta = .17$ как будто, мы заранее знаем, сколько нам ожидать фрода в выборке.

In [ ]:
def show_metrics(y_true, y_pred):
    return pd.Series(
        {
        "accuracy": accuracy_score(y_true, y_pred),
            
        "roc_auc": roc_auc_score(y_true, y_pred),
        
        "precision": precision_score(y_true, y_pred),# ratio tp / (tp + fp)
        
        "recall": recall_score(y_true, y_pred), # recall is the ratio tp / (tp + fn) 
        
        "f1_score": f1_score(y_true, y_pred),
        
        "f_0.2": fbeta_score(y_true, y_pred, beta=.17),
    }
)

## Обнаружение выбросов

In [ ]:
# author @artonson

def plot_clustering(model, data, size=100):
    def _expand(a, b, frac=.5, margin=1.):
        return a - abs(a) * frac - margin, b + abs(b) * frac + margin

    # Вспомогательная функция для рисования линий уровня и набора точек
    plt.figure(figsize=(9, 5))
    min_x, min_y = data.min(axis=0)
    max_x, max_y = data.max(axis=0)
    min_x, max_x = _expand(min_x, max_x)
    min_y, max_y = _expand(min_y, max_y)

    # создаём регулярную сетку для контуров
    all_x = np.linspace(min_x, max_x, num=size)
    all_y = np.linspace(min_y, max_y, num=size)
    XX, YY = np.meshgrid(all_x, all_y)
    test_data = np.c_[XX.ravel(), YY.ravel()]

    # опрашиваем предсказания модели
    try:
        predictions = model.decision_function(test_data).reshape(size, size)
        data_scores = model.predict(data)
        anomaly_scores = model.decision_function(data)

    except AttributeError:
        predictions = model._decision_function(test_data).reshape(size, size)
        data_scores = model._predict(data)
        anomaly_scores = model._decision_function(data)

    # создаём график контуров с заливкоц
    plt.contourf(all_x, all_y, predictions, cmap=plt.cm.coolwarm)

    # отображаем границу принятия решений
    threshold = anomaly_scores[data_scores==1.0].min()
    plt.contour(XX, YY, predictions, levels=[threshold], linewidths=1)

    # нарисуем точки выборки
    plt.scatter(data[:, 0], data[:, 1])

    axes = plt.gca()
    axes.set_xlim([min_x,max_x])
    axes.set_ylim([min_y,max_y])

    plt.show()
    plt.close()

Создаём несколько скоплений точек и подмешиваем аномальных точек

In [ ]:
from sklearn.datasets import make_blobs


def data_generator(n_samples=100, anomaly_fraction=0.1, n_features=2):
    n_anomaly = int(n_samples * anomaly_fraction)
    n_normal = n_samples - n_anomaly

    normal_data, _ = make_blobs(n_normal, n_features=n_features, centers=3)#какое это распределение?

    anomaly_data = np.random.rand(n_anomaly, n_features)

    nrm_min = normal_data.min(axis=0).reshape(1, -1)
    nrm_ptp = normal_data.ptp(axis=0).reshape(1, -1)
    anomaly_data = anomaly_data * nrm_ptp + nrm_min

    return np.concatenate([normal_data, anomaly_data], axis=0)

Создаём несколько скоплений точек, к которым подмешаны аномалии

In [ ]:
data_blobs = data_generator()

<br>

### Эллиптическая Огибающая

Метод **Elliptic Envelope** оценивая ковариацию данных предполагает, что 
* данные порожденные эллиптическим распределением

* аномальные точки находятся дальше от центра скопления, чем нормальные

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
model = EllipticEnvelope(assume_centered=False, contamination=0.1, random_state=42)

model.fit(data_blobs)

plot_clustering(model, data_blobs)

+ простой в использовании метод
+ порождает интерпретируемую границу
- годится только для распредений с одним цетром (одномодальных)

Посмотрим на его работу в обнаружении мошенничества

#### А как выгладит распредление наших переменых?

In [ ]:
model = EllipticEnvelope(random_state=42, contamination=0.02)

model.fit(train_X)

predictions_elliptic = -model.decision_function(test_X)

labels_elliptic = model.predict(test_X)

In [ ]:
max(-predictions_elliptic), min(-predictions_elliptic)

In [ ]:
labels_elliptic

In [ ]:
(labels_elliptic < 0)*1

In [ ]:
metrics_elliptic = show_metrics(test_y, (labels_elliptic < 0) * 1)
metrics_elliptic

<br>

### Изолирующий Лес (Isolation Forest)

* для каждого наблюдения рассчитывается средняя длина пути до него в случайном дереве
  * деревья со случайными разбиениями с целью изолировать наблюдения
  
* чем короче средняя длина пути тем проще описать наблюдение
  * тем в более изолированной части пространства оно находится

* чем больше разбиений нужно для наблюдения, нормальнее оно

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
model = IsolationForest(random_state=42)
model.fit(data_blobs)

plot_clustering(model, data_blobs)

Минусы: не различает скопления аномалий

In [ ]:
model = IsolationForest(random_state=42, contamination=0.0173)

model.fit(train_X)

predictions_isolation = -model.decision_function(test_X)

labels_isolation = model.predict(test_X)

## Выберем только тех, кого метод назвал аутлайерами

In [ ]:
test_X[(labels_isolation < 0).tolist()]

In [ ]:
metrics_isolation = show_metrics(test_y, (labels_isolation < 0) * 1)
metrics_isolation 

<br>

## Local Outlier Factor

Основан на наблюдении, что нормальные наблюдения имеют тенденцию **скапливаться**

* вводится показатель локальной плотности, обратно пропорциональный средним расстоянием до $k$ ближайших соседей

* попарно сравнивается с показателями соседей

* вычисляется отношение локальной аномальности

Метод: https://towardsdatascience.com/local-outlier-factor-for-anomaly-detection-cc0c770d2ebe

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
model = LocalOutlierFactor(novelty= True)
model.fit(data_blobs)

plot_clustering(model, data_blobs)

Минусы:
* подвержен проблеме "проклятия размерности", тк основан на расстояниях
* не может отличить скопления аномалий от нормальных точек

In [ ]:
model = LocalOutlierFactor()

model.fit(train_X)

predictions_lof = -model._decision_function(test_X)

labels_lof = -predictions_lof

In [ ]:
metrics_lof = show_metrics(test_y, (labels_lof < 0) * 1)
print( metrics_lof )

## One Class SVM

Основная идея -- отделить данные в спрямляющем пространстве **мягкой гиперплоскостью** от нуля

Решает задачу
\begin{aligned}
  & \underset{\rho, f\in \mathcal{H}}{\text{минимизировать}}
    & & \tfrac12 \|f\|^2 - \rho
        + \tfrac1{m \nu} \sum_{i=1}^m \max\bigl\{
            0, \rho - f(x_i) \bigr\}\,,
\end{aligned}

Визуализация одноклассового метода опорных векторов: http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/

Визуализация `gamma` для `rbf` ядра: https://bitquill.net/blog/quick-hack-visualizing-rbf-bandwidth/

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
model = OneClassSVM(nu=0.1, kernel='rbf', gamma=0.1)

model.fit(data_blobs)

plot_clustering(model, data_blobs)

Плюсы:
* непараметрический метод
* применим не только к объектам из $\mathbb{R}^n$ (линейного пространства)
  * ядра на строках, графах и пр.
* может быть полезным при разумном выборе ядра

Минусы:
* вычислительно затратен и плохо масштабируется

In [ ]:
train_X.shape

In [ ]:
model = OneClassSVM(random_state = 42, nu = 0.0173)

model.fit(train_X)

predictions_svm = model.decision_function(test_X)

labels_svm = model.predict(test_X)

In [ ]:
labels_svm

In [ ]:
metrics_svm = show_metrics(test_y, ~(labels_svm < 0) * 1)
print(metrics_svm )

<br>

## Бинарная классификация

Воспользуемся логистической регрессией

In [ ]:
from sklearn.linear_model import LogisticRegression


estimator = LogisticRegression(class_weight=None)

grid = {
    "C" : np.logspace(-3, +3, num=10)
}

Будем стараться честно построить модель

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer

f1_scorer = make_scorer(f1_score)

In [ ]:
from sklearn.model_selection import StratifiedKFold

st_kfold = StratifiedKFold(n_splits=5)

Оценим лог-регрессию с кросс валидацией

In [ ]:
cv_grid = GridSearchCV(estimator, grid, scoring=f1_scorer, cv=st_kfold, n_jobs=1)

cv_grid.fit(train_X, train_y) ;

Оценим метрики для самой лучшей модели по валидации

In [ ]:
logistic_naive = cv_grid.best_estimator_

logistic_naive_test_pred = logistic_naive.predict(test_X)

metrics_logistic_naive = show_metrics(test_y, logistic_naive_test_pred)

In [ ]:
metrics_logistic_naive

Можно ли улучшить `precision` и `recall`?

<br>

### Балансировка и Ресэмплинг

Функция потерь, взвешенная по меткам класса:
\begin{aligned}
  & \underset{\beta_0, \beta}{\text{минимизировать}}
    & & \tfrac12 \|\beta\|_2^2
        + C w_+ \sum_{i\colon y_i = +1} l\bigl(+1, f(x_i)\bigr)
        + C w_- \sum_{i\colon y_i = -1} l\bigl(-1, f(x_i)\bigr)
\end{aligned}

* дефолтный вариант $w_+ = w_- = 1$.

* сбалансированный вариант $w_+ = \tfrac{m}{2 n_+}$ и $w_- = \tfrac{m}{2 n_-}$.

В данном случае, мы делаем поправку на колличество представителей классов $n_ -$ и $n_+$

In [ ]:
estimator = LogisticRegression(class_weight = "balanced")

cv_grid = GridSearchCV(estimator, grid, scoring=f1_scorer, cv = st_kfold, n_jobs = 1)

cv_grid.fit(train_X, train_y) ;

In [ ]:
logistic_balanced = cv_grid.best_estimator_

logistic_balanced_test_pred = logistic_balanced.predict(test_X)

metrics_logistic_balanced = show_metrics(test_y, logistic_balanced_test_pred)

In [ ]:
metrics_logistic_balanced

#### Ресэмплинг

Сбалансируем классы в обучающей выборке добавлением наблюдений или изменением их веса.

In [ ]:
data.shape

In [ ]:
from sklearn.utils import check_random_state, safe_indexing


def undersample(X, y, ratio=20, pos_label=1, random_state=None):
    random_state = check_random_state(random_state)

    # отбрасываем случайную долю наблюдений доминирующего класса
    class_major_index = np.flatnonzero(y != pos_label)

    n_major = int(len(class_major_index) / ratio)
    class_major_index = random_state.permutation(class_major_index)
    class_major_index = class_major_index[:n_major]

    # выбираем все примеры минорного класса
    class_minor_index = np.flatnonzero(y == pos_label)

    # составляем новую (временную) обучающую выборку
    indices = np.r_[class_major_index, class_minor_index]

    return safe_indexing(X, indices), safe_indexing(y, indices)

Прореживание доминирующего класса может плохо обобщить из-за нехватки данных!

In [ ]:
estimator = LogisticRegression(class_weight=None)

Выделим из текущего трейна валидацоннный набор

In [ ]:
dev_X, val_X, dev_y, val_y = train_test_split(train_X, train_y, test_size=0.25,
                                              stratify=train_y, random_state=321)

Проведём валидацию модели с андерсэмплингом вручную

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.base import clone

n_resamples, ratio = 7, 10
par_grid = ParameterGrid(grid)

results_grid = []
for par in par_grid:

    results_resample = []
    for b in range(n_resamples):
        und_X, und_y = undersample(dev_X, dev_y, ratio, pos_label=1, random_state=None)

        cv_estimator = clone(estimator).set_params(**par)
        cv_estimator.fit(und_X, und_y)
        
        cv_val_pred = cv_estimator.predict(val_X)
        results_resample.append(show_metrics(val_y, cv_val_pred))

    results_resample = pd.concat(results_resample, axis=1).T

    results_grid.append((par, results_resample.mean().rename("mean")))

Выбираем $F_1$ метрики и находим наилучшую модель

In [ ]:
f1_scores = [(par, met["f1_score"]) for par, met in results_grid]

best_par_, _ = f1_scores[np.argmax([f1 for par, f1 in f1_scores])]

Делаем андерсэмплинг и обучаем модель заново

In [ ]:
und_X, und_y = undersample(train_X, train_y, ratio, pos_label=1, random_state=None)

logistic_undersample = clone(estimator).set_params(**best_par_)
logistic_undersample.fit(und_X, und_y)

Считаем метрики

In [ ]:
logistic_undersample_test_pred = logistic_undersample.predict(test_X)

metrics_logistic_undersample = show_metrics(test_y, logistic_undersample_test_pred)
print(metrics_logistic_undersample)

Сравним метрики

In [ ]:
all_metrics = pd.concat(dict([
    ("elliptic", metrics_elliptic),
    ("isolation", metrics_isolation),
    ("svm", metrics_svm),
    ("logistic_naive", metrics_logistic_naive),
    ("logistic_balanced", metrics_logistic_balanced),
    ("logistic_undersample", metrics_logistic_undersample)
]), axis=1)

all_metrics

##  Задание 1:
повторить эксперименты, имея 10% ключей для выборки

- произвести выбор лучшей стратегии детектирования аномалий на 5% ключей
- предсказать на остальных 5% ключей

In [ ]:
from sklearn.model_selection import train_test_split

train_X.reset_index(drop=True)

tt_split = train_test_split(full_X, full_y, test_size=0.1, stratify=full_y, random_state= 42)

train_X, test_X, train_y, test_y = tt_split 

In [ ]:
test_X.shape

#### Теперь разделим эту выборку на 2 половины `logistic_naive`.

In [ ]:
tt_split = train_test_split(test_X, test_y, test_size=0.5, stratify=test_y, random_state= 42)

train_5_X, test_5_X, train_5_y, test_5_y = tt_split 

In [ ]:
train_5_X.shape

#### Лучшие стратерии на всей выборке для нас были `IsolationForest` и `Logistic naive`

### Начнем с моделей обучения с учителем:
- заполним 90% датасета обучившись на 5% трейновой выборки, проверим результат на тесте 5%
- обучимся на 95% датасета и трейновой выборке, проверим результат на 5%

<br>